In [1]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

#import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

In [2]:
#funciones

def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets

Vamos a mapear los reads verdaderos de maiz a los genomas de la base de datos interna de referencia de _Clavibacter_ (BDIRC) 
0. Crear link simbolicos
1. Indexar los genomas de BDIRC con bowtie
2. Mapear con bowtie a esos genomas
3. Si queremos visualizar los reads podemos usar tablet, y necesitamos samtools para indexar el genoma y ver los reads alineados.
https://www.metagenomics.wiki/tools/bowtie2

In [2]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

A continuación creamos indices con para cada uno de los genomas refencia, para ello utilizamos el comando `bowtie2-build`. Los respectivos archivos de salida los almacenamos en la carpeta `index-bowtie`. Adicionalmente, creamos el archivo `list-genomes.txt` que contiene los nombre de cada archivo de salida del `bowtie2-build`.

In [3]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/index-bowtie/$id;
    echo "$id" >>list-genomes.txt
done

#Por ahora solo corre en bash





Settings:
  Output files: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genomas-clavi/clavi-genomes/Clavibacter_michiganensis_subsp_capsici_1101.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Total time for call to driver() for forward index: 00:00:00
Settings:
  Output files: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1106.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: 

Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Command: /opt/anaconda3/envs/metagenomics/bin/bowtie2-build-s --wrapper basic-0 genomas-clavi/clavi-genomes/Clavibacter_michiganensis_subsp_capsici_1101.fna genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101 
Deleting "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.3.bt2" file written during aborted indexing attempt.
Deleting "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.4.bt2" file written during aborted indexing attempt.
Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1106.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Comm

Lo siguiente es mapear los reads a los genomas de referencia, utilizamos `bowtie2` y obtenemos arhivos de salidas *.sam* que guardaremos en la carpeta `output-bowtie` 

#reference=subsp_capsici_1101  ; usar para extraer las lineas de archivo sed '1q;d' index-genomes.txt
for infile in ex-reads-clavi/fastq/fastq/capsicum/*-clav-1.fq
 do 
base=$(basename ${infile} -clav-1.fq)
dir=$(dirname ${infile} )
bowtie2 -x genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101 -1 ${dir}/${base}-clav-1.fq -2 ${dir}/${base}-clav-2.fq  --no-unal -p 12 -S genomas-clavi/clavi-genomes/out-bowtie/${base}.sam
done

In [4]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
bash 01-mapear-reads-a-referencia.sh insidiosus_CFBP_2404

(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a

Ahora corremos el scrip `01-mapear-reads-a-referencia.sh` por cada una de la lista de genomas que se encuentra en `index-genomes.txt´ 

In [ ]:
cat index-genomes.txt | while read line;
do
    bash 01-mapear-reads-a-referencia.sh $line
done

## Prueba 1: Utilizar solo 3 genomas
Ahora procedor hacer lo anterior son con 3 genomas, 1 de Clavibacter Michiganensis Michiganensis (cmm), y 2 de Michagenesis capsisi. 
Primero cremos los index



In [38]:
%%bash
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/genomas-prueba1/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia.txt
done

Settings:
  Output files: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genomas-clavi/clavi-genomes/genomas-prueba1/Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Total time for call to driver() for forward index: 00:00:00
Settings:
  Output files: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1106.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)

bash: line 1: /home/shaday/anaconda3/etc/profile.d/conda.sh: No such file or directory

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Command: /opt/anaconda3/bin/bowtie2-build-s --wrapper basic-0 genomas-clavi/clavi-genomes/genomas-prueba1/Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1

Ahora, hacemos el mapeo de los read simulados a cada genoma de refencia previamente indexado


In [48]:
%%bash
#source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia.txt | while read line;
do
bowtie2 -x output-tda/prueba-3genomas/index-bowtie/$line -U reads-sim1K.fastq.fastq --no-unal -p 12 -S output-tda/prueba1Kread/out-bowtie/Simulados-${line}.sam
done


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Use of uninitialized value $bt2_args[4] in join or string at /opt/anaconda3/bin/bowtie2 line 423.
Use of uninitialized value $bt2_args[5] in join or string at /opt/anaconda3/bin/bowtie2 line 423.
Use of uninitialized value in exists at /opt/anaconda3/bin/bowtie2 line 81.
Use of uninitialized value in exists at /opt/anaconda3/bin/bowtie2 line 81.
Use of uninitialized value $bt2_args[4] in join or string at /opt/anaconda3/bin/bowtie2 line 459.
Use of uninitialized value $bt2_args[5] in join or string at /opt/anaconda3/bin/bowtie2 line 459.
(ERR): Could not open output file 'output-tda/prueba1Kread/out-b

CalledProcessError: Command 'b'#source ~/anaconda3/etc/profile.d/conda.sh\nconda activate metagenomics\n\ncat list-genomasrefencia.txt | while read line;\ndo\nbowtie2 -x output-tda/prueba-3genomas/index-bowtie/$line -U reads-sim1K.fastq.fastq --no-unal -p 12 -S output-tda/prueba1Kread/out-bowtie/Simulados-${line}.sam\ndone\n'' returned non-zero exit status 13.

### Crear archivos .bam para visualizar y extraer información del match

In [6]:
%%bash
for infile in output-tda/prueba1Kread/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view ${infile} -o output-tda/prueba1Kread/bam/${base}.bam
    samtools sort ${infile} -o output-tda/prueba1Kread/sorted/${base}_sorted.sam
done

Crea un archivo *.txt con la salida mostrando el identificador y el read que identifico en el genoma referencia.

In [51]:
%%bash
for infile in output-tda/prueba1Kread/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/prueba1Kread/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/prueba1Kread/bam/${base}.txt
done

Obtener las columnas que nos interesan de los archivos .sam

In [52]:
%%bash
cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/prueba1Kread/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/prueba1Kread/tablas/${base}_extrac.txt
done


Importar tablas de mapeos


In [4]:
ruta_tablas = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/prueba1Kread/tablas/'
contenido = os.listdir(ruta_tablas)
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[16:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009478.1_0,0,NZ_CP048047.1,578115,42,150M,capsici_1106
1,NC_009480.1_1,16,NZ_CP048048.1,133457,42,150M,capsici_1106
2,NC_009478.1_1,0,NZ_CP048048.1,16945,38,150M,capsici_1106
3,NC_009479.1_1,16,NZ_CP048048.1,42991,42,150M,capsici_1106
4,NZ_CP048047.1_1,0,NZ_CP048047.1,27280,42,150M,capsici_1106
...,...,...,...,...,...,...,...
4397,NC_009478.1_993,0,NC_009479.1,12053,42,150M,michi_contigs
4398,NC_009479.1_993,0,NC_009479.1,1952,0,122M1D28M,michi_contigs
4399,NZ_CP048049.1_993,0,NC_009480.1,2365649,3,150M,michi_contigs
4400,NC_009480.1_994,0,NC_009480.1,81133,42,150M,michi_contigs


In [3]:
df.to_csv('MapeoReads-organismos.csv')

Creamos un diccionario para los organimos, y remplazamos en el dataframe


In [5]:
X=df['Organims'].unique().tolist()
len(X)
dic_Organisms={}
for k in range(len(X)):
    dic_Organisms[X[k]]="X{0}".format(k+1)
dic_Organisms
df2=df.replace({"Organims": dic_Organisms}) 

X2=df2['Organims'].unique().tolist() #lista de genomas 
A=powerset(X2)      #conjunto potencia
df2

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NC_009478.1_0,0,NZ_CP048047.1,578115,42,150M,X1
1,NC_009480.1_1,16,NZ_CP048048.1,133457,42,150M,X1
2,NC_009478.1_1,0,NZ_CP048048.1,16945,38,150M,X1
3,NC_009479.1_1,16,NZ_CP048048.1,42991,42,150M,X1
4,NZ_CP048047.1_1,0,NZ_CP048047.1,27280,42,150M,X1
...,...,...,...,...,...,...,...
4397,NC_009478.1_993,0,NC_009479.1,12053,42,150M,X3
4398,NC_009479.1_993,0,NC_009479.1,1952,0,122M1D28M,X3
4399,NZ_CP048049.1_993,0,NC_009480.1,2365649,3,150M,X3
4400,NC_009480.1_994,0,NC_009480.1,81133,42,150M,X3


Vamos a crear un diccionario que contenga los reads y los organismos a los cuales son mapeados

In [6]:
dic={}
l=0
Reads=df2['Qname'].unique().tolist()
for k in Reads:
    valores=list(df2.loc[df2.Qname==k].iloc[:,6])
    dic[k]=valores


La función ´Sy´ calcula los reads que son mapeados solamente a $Y\in2^X$.


In [7]:
def Sy(dic,X):
    dict_items=dic.items()
    i=0
    for k in X:
        myValue=k
        globals()["Y{}".format(i)] =[key for key,value in dict_items if value==myValue]
        mykey=[key for key,value in dict_items if value==myValue]
        i+=1
    return Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7
    
    

In [9]:
# Calculamos SY para cada Y en 2**X
SY=[]
SY= Sy(dic,A)


In [10]:
#Exportamos Sy a txt
for k in range(len(SY)):
    with open('output-tda/prueba1Kread/SyReads/S_y{0}.txt'.format(k), 'w') as f:
        for line in SY[k]:
            f.write(f"{line}\n")

In [7]:
%%bash
for infile in output-tda/prueba1Kread/sorted/*.sam
do 
	base=$(basename ${infile} .sam)
    head -4 ${infile} > output-tda/prueba1Kread/sam_extract/${base}-S_y2.bam  #get the head the .sam file's
	samtools view ${infile} | grep -f output-tda/prueba1Kread/SyReads/S_y2.txt >> output-tda/prueba1Kread/sam_extract/${base}-S_y2.bam # append the read selected 
    done

In [46]:
%%bash 
output-tda/prueba1Kread/SyReads/

/home/shaday/GIT/clavibacter/scripts-tda
